In [154]:
def recalc_micas(path,sheet_name,site_occupancy):
    """
    READ ME

    This will recalculate mica analysis on the basis of 22 oxygens per formula
    Assumptions about site occupancy will have to be chosen by the user. Selections are noted below.
    F, Cl, and SO3 (or SO2 for that matter) are considered if they are measured. Otherwise they are ignored and the user can decide if they want to
        calculate these by difference.
    Al(IV) and Al(III) are calculated by difference with Si

    Inputs:
        path = some path name to your file. This must be input in parentheses and end in .xlsx
        sheet_name = input the sheet name of that excel file where your data lives. This must also be in parentheses.
             Your file headers must be in all lower case. For instance, SiO2 must be sio2, etc
            Your first column must be the name that you gave your spot when measuring
        Site Occupancy:
            ferrous: Assumes all Fe is Fe2+
            ferric: Assumes all Fe is Fe3+
            no phenogite: Places a restriction on the Si:Al ratio (Si6:Al2). This is not (generally) recommended.
            no tetra Al: Forces all Al out of tetrahedral coordination. This is not recommended.
            Al in dioctahedral: As much Al as possible to the Dioctahedral site.
            no trioctahedral subs: All Mg=6. Probably ok for Talc...
            cctahedral sites full: Sum Mg=7 (supposedly)
            A site full: K+Na+Ca=1 (supposedly)
    """

    #import dependables
    import pandas as pd
    import numpy as np
    import math
    from statistics import stdev
    path = '/Users/ctlewis/Documents/Projects/Caspana_052921/Phases/Chuck_biotite_091719.xlsx'
    sheet_name = 'py_code'

    site_occupancy = 'A site full'

    df = pd.read_excel(path,sheet_name)
    #set up a general dataframe that has a number of oxides, their number of O2-, the cation:anion, and the molecular weights
    oxides = pd.DataFrame([[60.0855,101.963,94.1966,56.087,79.867,70.938,71.845,159.692,61.9796,40.305,141.9476,151.990,74.693,64.066,153.326,149.881,265.81,81.379,18.998,35.453,80.063],
                          [2,3,1,1,2,1,1,3,1,1,5,3,1,2,1,3,5,1,1,1,3],
                          [1/2,2/3,2,1,1/2,1,1,2/3,2,1,2/5,2/3,1,1/2,1,2/3,2/5,1,0.5,0.5,1/3]],
                      columns=['sio2','al2o3','k2o','cao','tio2','mno','feo','fe2o3','na2o','mgo','p2o5','cr2o3','nio','so2','bao','v2o3','nb2o5','zno','f','cl','so3'])
    #pull out the column with strings
    spots = df.iloc[:,0]
    #pull out all the columns with data in them
    data = df.iloc[:,1:]
    #set any negative numbers = 0
    data[data < 0] = 0
    #put together the measured data and the general oxides dataframe, dropping any columns from 'oxides' that does not exist in the measured dataset 
    if site_occupancy == 'ferric':
        data['fe2o3'] = data['feo']*(159.692/(2*71.845))
        #anions.drop('feo_molar proportion',axis=1)
        data['feo'] = 0
    else:
        pass

    newframe = pd.concat([data,oxides], join='outer').fillna(0)

    #the line below does a few things:
    #1. gives you the molar proportion of oxides by dividing measurements by molecular weights
    #2. reorganizes the dataframes to retain the oxides dataframe in a neat manner
    #3. adds a suffix for the eventual output dataframe accordingly
    newframe = pd.concat([newframe.iloc[:-3,:].div(newframe.iloc[-3,:]),newframe.iloc[-3:]]).reset_index().drop('index',axis=1).add_suffix('_molar proportion')

    #multiply by the number of oxygens per oxide in order to get the atomic proportion of oxygen
    atomico2 = newframe.iloc[:-3,:].multiply(newframe.iloc[-2,:])
    #add up the atomic proportion of oxygen. This helps you normalize your measurement in a step below
    atomico2['number o2-'] = atomico2.sum(axis=1)
    #make a normalizing factor for each analysis based on an ideal number of oxygen with respect to what you actually measured
    atomico2['normalizing factor'] = 22 / atomico2['number o2-']
    #put the oxides information back in for ease of calculating and cleanliness
    atomico2 = pd.concat([atomico2,newframe.iloc[-3:]])

    #get the amount of negative charge that is required for each of the cations you measured. This is effectively how your ideal number of negative charges (which you've obtained by normalizing) are distributed about your mineral site based on the cations that are actually measureable.
    anions = atomico2.iloc[:-3,:-2].multiply(atomico2.iloc[:-3,-1],axis=0)
    #sum the total number of negative charges. This should sum up to the numerator you have used above. For the case of olivine, this would be 4.
    anions['negative charge total'] = anions.sum(axis=1)
    #put the oxides information back in for ease of calculating and cleanliness
    anions = pd.concat([anions,newframe.iloc[-3:]])
    #change the suffix at the end of this series of columns for outputting later
    anions.columns = anions.columns.str.replace('_molar proportion','_charge distribution')

    #multiply the distributed oxygen charge by the ratio of cation to anion. A simple unit analysis should guide your intution
    cations = anions.iloc[:-3,:-1].multiply(anions.iloc[-1,:-1])

    if (site_occupancy == 'ferric' or site_occupancy == 'ferrous'):
        #sum the number of cations. This should be pretty close to your mineral stoichiometry.
        cations['sum cations'] = cations.sum(axis=1)
        #change the suffix for hte purposes of outputting later
        cations.columns = cations.columns.str.replace('_charge distribution','_as cation')
        #calculate the normalization needed to get an ideal number of cations. For pyroxenes this is done to get Fe2+ and Fe3+. There are some critical assumptions here and you should always keep the limitations of your measurements in mind
        cations['fe/(fe+mg)'] = cations['feo_as cation'] / (cations['feo_as cation'] + cations['mgo_as cation'])
        cations['Al(IV)'] = [i if 8-x>i else 8-x for i,x in zip(cations['al2o3_as cation'],cations['sio2_as cation'])]
        cations['Al(III)'] = cations['al2o3_as cation'] - cations['Al(IV)']
        mica_recalc = pd.concat([spots,data,atomico2,anions,cations],axis=1).fillna(0)
    else:
        pass

    if (site_occupancy == 'ferric' or site_occupancy == 'ferrous'):
        pass
    elif site_occupancy == 'no phengite':
        cations.columns = cations.columns.str.replace('_charge distribution','_cations unnormalized')
        cations['sum cations'] = cations.sum(axis=1)
        cations['cation norm factor'] = 6 / cations['sio2_cations unnormalized']
        #get the normalized number of cations, based on an ideal number of 4
        cations_norm = cations.iloc[:,:].multiply(cations.iloc[:,-1],axis=0).drop('sum cations',axis=1)
        cations_norm.columns = cations_norm.columns.str.replace('_cations unnormalized','_cations normalized')
        #sum them
        #cations_norm['sum normalized cations'] = cations_norm.sum(axis=1)

    elif site_occupancy == 'no tetra Al':
        cations.columns = cations.columns.str.replace('_charge distribution','_cations unnormalized')
        cations['sum cations'] = cations.sum(axis=1)
        cations['cation norm factor'] = 8 / cations['sio2_cations unnormalized']
        cations_norm = cations.iloc[:,:].multiply(cations.iloc[:,-1],axis=0).drop('sum cations',axis=1)
        cations_norm.columns = cations_norm.columns.str.replace('_cations unnormalized','_cations normalized')
        #cations_norm['sum normalized cations'] = cations_norm.sum(axis=1)

    elif site_occupancy == 'Al in dioctahedral':
        cations.columns = cations.columns.str.replace('_charge distribution','_cations unnormalized')
        cations['sum cations'] = cations.sum(axis=1)
        cations['cation norm factor'] = 12 / (cations['sio2_cations unnormalized'] + cations['tio2_cations unnormalized'] + cations['al2o3_cations unnormalized'])
        cations_norm = cations.iloc[:,:].multiply(cations.iloc[:,-1],axis=0).drop('sum cations',axis=1)
        cations_norm.columns = cations_norm.columns.str.replace('_cations unnormalized','_cations normalized')
        cations_norm['sum normalized cations'] = cations_norm.sum(axis=1)

    elif site_occupancy == 'no trioctahedral subs':
        cations.columns = cations.columns.str.replace('_charge distribution','_cations unnormalized')
        cations['sum cations'] = cations.sum(axis=1)
        cations['cation norm factor'] = 12 / (cations['sio2_cations unnormalized'] + cations['tio2_cations unnormalized'] + cations['al2o3_cations unnormalized'] + cations['feo_cations unnormalized'] + 
                                             cations['mno_cations unnormalized'] + cations['mgo_cations unnormalized'])
        cations_norm = cations.iloc[:,:].multiply(cations.iloc[:,-1],axis=0).drop('sum cations',axis=1)
        cations_norm.columns = cations_norm.columns.str.replace('_cations unnormalized','_cations normalized')
        cations_norm['sum normalized cations'] = cations_norm.sum(axis=1)

    elif site_occupancy == 'octahedral site full':
        cations.columns = cations.columns.str.replace('_charge distribution','_cations unnormalized')
        cations['sum cations'] = cations.sum(axis=1)
        cations['cation norm factor'] = 14 / (cations['sio2_cations unnormalized'] + cations['tio2_cations unnormalized'] + cations['al2o3_cations unnormalized'] + cations['feo_cations unnormalized'] + 
                                              cations['mno_cations unnormalized'] + cations['mgo_cations unnormalized'])
        cations_norm = cations.iloc[:,:].multiply(cations.iloc[:,-1],axis=0).drop('sum cations',axis=1)
        cations_norm.columns = cations_norm.columns.str.replace('_cations unnormalized','_cations normalized')
        cations_norm['sum normalized cations'] = cations_norm.sum(axis=1)

    elif site_occupancy == 'A site full':
        cations.columns = cations.columns.str.replace('_charge distribution','_cations unnormalized')
        cations['sum cations'] = cations.sum(axis=1)
        cations['cation norm factor'] = 2 / (cations['cao_cations unnormalized'] + cations['k2o_cations unnormalized'] + cations['na2o_cations unnormalized'])
        cations_norm = cations.iloc[:,:].multiply(cations.iloc[:,-1],axis=0).drop('sum cations',axis=1)
        cations_norm.columns = cations_norm.columns.str.replace('_cations unnormalized','_cations normalized')
        cations_norm['sum normalized cations'] = cations_norm.sum(axis=1)
    else:
        pass

    if (site_occupancy == 'ferric' or site_occupancy == 'ferrous'):
        pass
    else:
        anions_norm = cations_norm.drop('cation norm factor',axis=1)
        anions_norm.columns = anions_norm.columns.str.replace('_cations normalized','')
        anions_norm = pd.concat([anions_norm,oxides], join='inner')
        anions_norm = anions_norm.iloc[:-3,:].divide(anions_norm.iloc[-1,:],axis=1)
        anions_norm['sum normalized anions'] = anions_norm.sum(axis=1)
        anions_norm['Needed O2'] = 22 - anions_norm['sum normalized anions']



    if (site_occupancy == 'ferric' or site_occupancy == 'ferrous'):
        pass
    elif site_occupancy =='no phengite':
        cations_final = cations_norm.drop(['al2o3_cations normalized','feo_cations normalized','cation norm factor','sum normalized cations'],axis=1)
        cations_final['Fe3+'] = [x*2 if (x>0 and i>x) else 0 for x,i in zip(anions_norm['Needed O2'],anions_norm['feo'])]
        cations_final['Fe2+'] = [i-x if x>0 else i for x,i in zip(cations_final['Fe3+'],anions_norm['feo'])]
        cations_final['Al(IV)'] = [1 if x>1 else x for x in cations_norm['al2o3_cations normalized']]
        cations_final['Al(III)'] = cations_norm['al2o3_cations normalized'] - cations_final['Al(IV)']
        cations_final['sum cations'] = cations_final.sum(axis=1)
        cations_final.columns = cations_final.columns.str.replace('_cations normalized','_as cation')

    elif site_occupancy == 'no tetra Al':
        cations_final = cations_norm.drop(['al2o3_cations normalized','feo_cations normalized','cation norm factor','sum normalized cations'],axis=1)
        cations_final['Fe3+'] = [x*2 if (x>0 and i>x) else 0 for x,i in zip(anions_norm['Needed O2'],anions_norm['feo'])]
        cations_final['Fe2+'] = [i-x if x>0 else i for x,i in zip(cations_final['Fe3+'],anions_norm['feo'])]
        cations_final['Al(IV)'] = 0
        cations_final['Al'] = cations_norm['al2o3_cations normalized']
        cations_final['sum cations'] = cations_final.sum(axis=1)
        cations_final.columns = cations_final.columns.str.replace('_cations normalized','_as cation')

    elif site_occupancy == 'Al in dioctahedral':
        cations_final = cations_norm.drop(['feo_cations normalized','cation norm factor','sum normalized cations'],axis=1)
        cations_final['Fe3+'] = [x*2 if (x>0 and i>x) else 0 for x,i in zip(anions_norm['Needed O2'],anions_norm['feo'])]
        cations_final['Fe2+'] = [i-x if x>0 else i for x,i in zip(cations_final['Fe3+'],anions_norm['feo'])]
        cations_final['sum cations'] = cations_final.sum(axis=1)
        cations_final.columns = cations_final.columns.str.replace('_cations normalized','_as cation')

    elif site_occupancy == 'no trioctahedral subs':
        cations_final = cations_norm.drop(['al2o3_cations_normalized','feo_cations normalized','cation norm factor','sum normalized cations'],axis=1)
        cations_final['Fe3+'] = [x*2 if (x>0 and i>x) else 0 for x,i in zip(anions_norm['Needed O2'],anions_norm['feo'])]
        cations_final['Fe2+'] = [i-x if x>0 and i-x>0 else i for x,i in zip(cations_final['Fe3+'],anions_norm['feo'])]
        cations_final['Al(IV)'] = [1 if x>1 else x for x in cations_norm['al2o3_cations normalized']]
        cations_final['Al(III)'] = cations_norm['al2o3_cations normalized'] - cations_final['Al(IV)']
        cations_final['sum cations'] = cations_final.sum(axis=1)
        cations_final.columns = cations_final.columns.str.replace('_cations normalized','_as cation')

    elif site_occupancy == 'octahedral site full':
        cations_final = cations_norm.drop(['al2o3_cations normalized','feo_cations normalized','cation norm factor','sum normalized cations'],axis=1)
        cations_final['Fe3+'] = [x*2 if (x>0 and i>x) else 0 for x,i in zip(anions_norm['Needed O2'],anions_norm['feo'])]
        cations_final['Fe2+'] = [i-x if x>0 and i-x>0 else i for x,i in zip(cations_final['Fe3+'],anions_norm['feo'])]
        cations_final['Al(IV)'] = [1 if x>1 else x for x in cations_norm['al2o3_cations normalized']]
        cations_final['Al(III)'] = cations_norm['al2o3_cations normalized'] - cations_final['Al(IV)']
        cations_final['sum cations'] = cations_final.sum(axis=1)
        cations_final.columns = cations_final.columns.str.replace('_cations normalized','_as cation')

    elif site_occupancy == 'A site full':
        cations_final = cations_norm.drop(['al2o3_cations normalized','feo_cations normalized','cation norm factor','sum normalized cations'],axis=1)
        cations_final['Fe3+'] = [x*2 if (x>0 and i>x) else 0 for x,i in zip(anions_norm['Needed O2'],anions_norm['feo'])]
        cations_final['Fe2+'] = [i-x if x>0 and i-x>0 else i for x,i in zip(cations_final['Fe3+'],anions_norm['feo'])]
        cations_final['Al(IV)'] = [1 if x>1 else x for x in cations_norm['al2o3_cations normalized']]
        cations_final['Al(III)'] = cations_norm['al2o3_cations normalized'] - cations_final['Al(IV)']
        cations_final['sum cations'] = cations_final.sum(axis=1)
        cations_final.columns = cations_final.columns.str.replace('_cations normalized','_as cation')

    cations_final['Fe2+/(Fe2++mg)'] = cations_final['Fe2+'] / (cations_final['Fe2+'] + cations_final['mgo_as cation'])

    mica_recalc = pd.concat([spots,data,atomico2,anions,cations,cations_final],axis=1).fillna(0)
    cations_final.head()

    output = mica_recalc.to_excel('mica_recalculated.xlsx')

    return output